In [ ]:
import os
import configparser

from langchain_groq import ChatGroq
from langchain_cohere import ChatCohere

from langchain_core.messages import HumanMessage, SystemMessage

config = configparser.ConfigParser()#creating instance of ConfigParser()-helps to read/write config files
config.read('../config.ini')
groq = config['groq']#extracting API keys from config.ini file
cohere = config['cohere']

os.environ['GROQ_API_KEY'] = groq.get('GROQ_API_KEY')#setting environment variables
os.environ['COHERE_API_KEY'] = cohere.get('COHERE_API_KEY')


from langchain_core.output_parsers import StrOutputParser#to parse output of LLM into meaningful sentence
from langchain_core.prompts import ChatPromptTemplate #import the Class for creating a prompt

parser = StrOutputParser()#to create instance 



In [5]:
model = ChatCohere(model="command-r-plus")#using model ChatCohere , trying to use without giving any input to model to write script
chain = model | parser #model output is passed to parser 
response = chain.invoke([SystemMessage("You will be given actor, theme and location. your task is to create the the movie script around actor, location and theme."),
HumanMessage(f"Actor for movie actor, Theme for movie , Location for movie.")])#to execute chain of operations 
print(response)

**Actor:** Leonardo DiCaprio 

**Theme:** Time Travel 

**Location:** Ancient Egypt 

## Movie Title: Sands of Time 

### Overview: 

'Sands of Time' is an epic adventure film that follows the journey of Jack Walker (Leonardo DiCaprio), a renowned archaeologist and time traveler, as he finds himself in ancient Egypt. Walker must navigate the treacherous landscape, decipher ancient mysteries, and confront powerful enemies as he attempts to uncover the secrets of a long-lost civilization. 

### Plot: 

The movie opens with a breathtaking aerial view of the vast Egyptian desert. We find Jack Walker and his team of archaeologists camping at the base of a towering pyramid. They have just discovered a long-forgotten tomb, and as they explore its depths, they stumble upon a mysterious artifact—an ancient hourglass filled with shimmering sand. 

Walker, fascinated by the artifact, realizes that it holds more than just historical value. Through his research, he discovers that the hourglass is a

In [7]:
from langchain_core.runnables import RunnableSequence #RunnableSequence helps to run series of task one after other, output of one task is passed as input to other task
model = ChatCohere(model="command-r-plus")
chain = RunnableSequence(model, parser)#output of model is passed to parser
import act_loc_th as atl #file where randomly selecting actor, location  and theme is defined
actor = atl.actor_choice() # rnadomly choosing from list of actors.
theme = atl.theme_choice() #randomly choosing  from list of themes.
location = atl.location_choice() #randomly choosing from list of locations.
print(str(actor),str(theme),str(location))#printing randomly selected variables
messages = [
    SystemMessage("You will be given actor, theme and location. your task is to create the the movie script around actor, location and theme."),
HumanMessage(f"Actor for movie = {actor}, Theme for movie = {theme}, Location for movie = {location}.")
]
result=chain.invoke(messages)
print(result) #using ChatCohere model 

Amitabh Bachan Drama new york
Title: "The Unforgiving City"

**Scene 1: The Streets of New York**

The bustling streets of New York City set the stage for our story. We find ourselves in the heart of Manhattan, where the bright lights and tall buildings contrast the dark underbelly of the city. 

Our protagonist, Raj Malhotra (played by Amitabh Bachan), is a middle-aged Indian man with a weary look on his face. He walks with a slight limp, a reminder of past troubles. Raj is a former businessman who has fallen on hard times, having lost his fortune and reputation due to a scandal that ruined his life. 

As he makes his way through the busy streets, he passes by a group of wealthy socialites stepping out of a luxurious car. Their laughter and expensive clothing remind Raj of the life he once had. He feels a pang of bitterness and continues on his way. 

**Scene 2: The Humble Abode**

Raj enters his modest apartment, located in a less glamorous part of town. The place is sparsely furnish

In [ ]:
# import the chat history classes
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
import asyncio # library for writing code that interacts with DB, network calls etc. 

#Create a store in memory
store = InMemoryChatMessageHistory()

# Lets define a function that gets messages from store
async def getMessage():
    await asyncio.sleep(2) # this will mimic a read from DB
    print("Messages retrieved from DB")
    return await store.aget_messages()

store.add_message(HumanMessage(f"Actor for movie = {actor}, Theme for movie = {theme}, Location for movie = {location}."))
messages = await(getMessage())
response = model.invoke(messages) # asyncio has runners for coroutines, context managers etc. 
print(response.content) # note that our first message is safely in the store
# lets add the message returned by the model to the store

store.add_message(SystemMessage(response.content))
store.add_message(HumanMessage('add Ema watson as co-actress for the movie'))
messages = await(getMessage())
print(messages) # check all the message are in store.
response = model.invoke(messages)
print(response.content) # Notice that the reponse now takes into account earlier interactions also.

store.add_message(HumanMessage('Breif the script'))
messages = await(getMessage())
response = model.invoke(messages)
print(response.content)

script_customise=input('would you like change actor, actress, theme or location?(type yes to customise.)').lower()
if script_customise=='yes' or script_customise=='y':
    a=input("enter new actor/actress: ")
    l=input("enter new location: ")
    t=input("enter new theme: ")
    store.add_message(HumanMessage(content=f'write a movie script for new actor {a}, new location {l}, new theme {t}'))
    messages = await(getMessage())
    response = model.invoke(messages)
    print(response.content)
           

Messages retrieved from DB
Sure! Here is a movie idea based on the given parameters:

**Movie Title:** "The Big Apple Dream"

**Plot:**

Amitabh Bachan stars as Raj Malhotra, an Indian immigrant who arrives in New York City chasing the American dream. He finds himself in the vibrant and diverse neighborhood of Jackson Heights, Queens, where he befriends a local café owner, Maya (played by Priyanka Chopra), who helps him navigate his new life in the city.

Raj starts working as a cab driver, and through his interactions with passengers from all walks of life, he begins to understand the complexities and challenges of life in the big city. He witnesses the stark contrast between the rich and the poor, the struggle for survival, and the pursuit of happiness and success.

As Raj becomes more entangled in the fabric of New York, he encounters a range of characters, including a wealthy Wall Street businessman, a struggling artist, and a homeless veteran. Each interaction teaches him valuable

In [10]:
from langchain_core.runnables.history import RunnableWithMessageHistory
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:  # If a new session then create a new memory store.
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

config = {'configurable': {"session_id": "abc2"}}
withHistory = RunnableWithMessageHistory(model, get_session_history)

import act_loc_th as atl #file where randomly selecting actor, location  and theme is defined
actor = atl.actor_choice() # rnadomly choosing from list of actors.
theme = atl.theme_choice() #randomly choosing  from list of themes.
location = atl.location_choice() #randomly choosing from list of locations.
print(str(actor),str(theme),str(location))#printing randomly selected variables

messages = [
    SystemMessage("You will be given actor, theme and location. your task is to create the the movie script around actor, location and theme."),
    HumanMessage(f"Actor for movie = {actor}, Theme for movie = {theme}, Location for movie = {location}.")
]

response = withHistory.invoke(messages,config=config)
print(response.content) 

script_customise=input('would you like change actor, actress, theme or location?(type yes to customise.)').lower()
if script_customise=='yes' or script_customise=='y':
    a=input("enter new actor/actress: ")
    l=input("enter new location: ")
    t=input("enter new theme: ")
    response = withHistory.invoke([HumanMessage(content=f'edit the movie script for new actor {a}, new location {l}, new theme {t}')], config=config)

print(response.content)

Mindy Kalling Drama Matamata
Title: "A River Runs Through Us"

**Scene 1: Matamata, New Zealand**

The camera pans over the picturesque town of Matamata, nestled amidst rolling hills and lush green pastures. We see a vibrant downtown area with quaint shops and cafes, and a bustling community. 

Cut to: A busy coffee shop, where we find our protagonist, Sara (played by Mindy Kalling), a talented but struggling novelist, typing away on her laptop. She sips her coffee and glances around the room, taking in the familiar faces and the cozy ambiance. 

Sara (voiceover): "Matamata. A town that feels like a warm hug. I've lived here my entire life, and these streets, these people, they're a part of me. But something was missing. I felt stagnant, my creative well running dry." 

A friend, Jane, approaches and sits down with a cheerful smile. 

Jane: "Hey, Sara! How's the novel coming along? You're always holed up here, typing away. I hope you're taking some time to enjoy life, too." 

Sara: "It